In [29]:
# scratch messing around with pushift with psaw: https://github.com/dmarx/psaw
# check out this also: https://www.reddit.com/r/pushshift/comments/8h31ei/documentation_pushshift_api_v40_partial/
# Can also just use Google's BigQuery: https://pushshift.io/using-bigquery-with-reddit-data/
# Check here from some updates to syntax: https://stackoverflow.com/questions/52068129/bigquery-and-pushshift-time-range-decorator-is-not-supported-on-partitioned-tab
# https://stackoverflow.com/questions/53732383/how-to-use-legacysql-to-query-table-with-partition-column
from psaw import PushshiftAPI
import json
api = PushshiftAPI()

In [44]:
# The `search_comments` and `search_submissions` methods return generator objects
gen = api.search_submissions(limit=100)

# next(gen)
results = list(gen)

In [107]:
### not using a wrapper 
### taken from: https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
### and: https://medium.com/@RareLoot/using-pushshifts-api-to-extract-reddit-submissions-fb517b286563
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta
import time


# takes URI and handles the https request/response
# if the request fails, tries max 5 times, waiting a second after each
def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)


## builds a URL request and returns the data, similar to above but I think more useful for this
# uses 500 max size and no query
def getPushshiftData(after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?&size=500&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)+'&metadata=true'
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data

# function for extracting key features of submission
def collectSubData(subm, subStats):
    subData = {}
    subData['title'] = subm['title']
    subData['url'] = subm['url']
    try:
        subData['flair'] = subm['link_flair_text']
    except KeyError:
        subData['flair'] = "NaN"    
    subData['author'] = subm['author']
    subData['sub_id'] = subm['id']
    subData['score'] = subm['score']
    subData['created'] = datetime.fromtimestamp(subm['created_utc'])
    subData['numComms'] = subm['num_comments']
    subData['permalink'] = subm['permalink']
#     subData['stickied'] = subm['stickied']
#     subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    subStats[subm['id']] = subData


# getting 500 posts from r/science
# resp = make_request('https://api.pushshift.io/reddit/comment/search?size=500&subreddit=science')
# print(len(resp['data']))



In [97]:
# same thing using the other function
before = "1538352000" #October 1st
after = "1104537600"  #January 1st, 2005 (before reddit was founded)
sub_dict = {}

resp = getPushshiftData(after, before, 'science')
print(resp.keys())
resp_data = resp['data']

print(resp['metadata'])
for s in resp_data:
    collectSubData(s, sub_dict)

# print(len(resp_data))
# print(sub_dict)

https://api.pushshift.io/reddit/search/submission/?&size=500&after=1104537600&before=1538352000&subreddit=science&metadata=true
dict_keys(['data', 'metadata'])
{'after': 1104537600, 'agg_size': 100, 'api_version': '3.0', 'before': 1538352000, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['science']}}, {'range': {'created_utc': {'gt': 1104537600}}}, {'range': {'created_utc': {'lt': 1538352000}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 179.04, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1104537600}}}, {'range': {'created_utc': {'lt': 1538352000}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['science'], 'timed_out': False, 'total_results': 709312}


In [104]:
### Returns all subreddit submissions (not comments)
def get_all_submissions(after, before, sub):
    subCount = 0
    sub_dict = {}

    resp = getPushshiftData(after, before, sub)
    print(resp['metadata'])
    data = resp['data']
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:
            collectSubData(submission, sub_dict)
            subCount+=1
        # Calls getPushshiftData() with the created date of the last submission
        print(len(data))
        print(str(datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']
        resp = getPushshiftData(after, before, sub)
        print(resp['metadata'])
        data = resp['data']


    print(len(data))
    print(subCount)
    print(str(len(sub_dict)) + " submissions have added to list")
    print("1st entry is:")
    print(list(sub_dict.values())[0]['title'] + " created: " + str(list(sub_dict.values())[0]['created']))
    print("Last entry is:")
    print(list(sub_dict.values())[-1]['title'] + " created: " + str(list(sub_dict.values())[-1]['created']))
    
    return sub_dict



In [105]:
### Testing from Chenhao's arguement paper: https://chenhaot.com/data/cmv/README.txt -- -- currently numbers are not matching up
sub='ChangeMyView'

#before and after dates - after will keep on getting pushed back
before = "1430956800" #05/07/2015 - Taken from paper
after = "1356998400"  #01/01/2013 - Taken from paper

submissions = get_all_submissions(after, before, sub)


https://api.pushshift.io/reddit/search/submission/?&size=500&after=1356998400&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1356998400, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1356998400}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 179.76, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1356998400}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 42453}
500
2013-03-29 15:33:19
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1364596399&before=

{'after': 1370937751, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1370937751}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 194.4, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1370937751}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 37953}
500
2013-06-17 11:46:01
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1371494761&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1371494761, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es

{'after': 1377016010, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1377016010}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 159.35, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1377016010}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 33453}
500
2013-08-26 14:42:30
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1377553350&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1377553350, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'e

{'after': 1383378623, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1383378623}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 239.65, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1383378623}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 28953}
500
2013-11-11 06:27:27
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1384180047&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1384180047, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'e

{'after': 1389902209, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1389902209}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 244.4, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1389902209}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 24453}
500
2014-01-24 16:04:44
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1390608284&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1390608284, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es

{'after': 1395929464, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1395929464}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 252.97, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1395929464}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 19953}
500
2014-04-01 19:35:38
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1396406138&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1396406138, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'e

{'after': 1402093152, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1402093152}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 285.24, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1402093152}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 15453}
500
2014-06-15 12:30:15
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1402860615&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1402860615, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'e

{'after': 1409503845, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1409503845}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 278.22, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1409503845}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 10953}
500
2014-09-10 09:14:30
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1410365670&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1410365670, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'e

{'after': 1418166309, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1418166309}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 275.4, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1418166309}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 6453}
500
2014-12-20 19:42:12
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1419133332&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1419133332, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_

{'after': 1427254663, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['changemyview']}}, {'range': {'created_utc': {'gt': 1427254663}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 314.27, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1427254663}}}, {'range': {'created_utc': {'lt': 1430956800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['ChangeMyView'], 'timed_out': False, 'total_results': 1953}
500
2015-04-04 08:56:10
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1428162970&before=1430956800&subreddit=ChangeMyView&metadata=true
{'after': 1428162970, 'agg_size': 100, 'api_version': '3.0', 'before': 1430956800, 'es

In [84]:
len(submissions)

42453

In [85]:
### Testing from Tims's pizza paper -- currently numbers are not matching up
sub='Random_Acts_Of_Pizza'

#before and after dates - after will keep on getting pushed back
before = "1380412800" #09/29/2013 - Taken from paper
after = "1291766400"  #12/08/2010 - Taken from paper

submission_pizza = get_all_submissions(after, before, sub)

https://api.pushshift.io/reddit/search/submission/?&size=500&after=1291766400&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1291766400, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1291766400}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 274.21, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1291766400}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 43541}
500
2011-06-15 13:49:03
https://api.pushshift.io/reddit/search/submission/?&size=500&

{'after': 1312077648, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1312077648}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 186.03, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1312077648}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 39041}
500
2011-08-04 15:34:09
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1312497249&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1312497249, 'agg_size': 100, 'api_version': '3.0', 

{'after': 1316837065, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1316837065}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 232.62, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1316837065}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 34541}
500
2011-09-30 19:08:23
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1317434903&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1317434903, 'agg_size': 100, 'api_version': '3.0', 

{'after': 1323213267, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1323213267}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 205.75, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1323213267}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 30041}
500
2011-12-14 18:43:33
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1323917013&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1323917013, 'agg_size': 100, 'api_version': '3.0', 

{'after': 1331099623, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1331099623}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 343.77, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1331099623}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 25541}
500
2012-03-16 13:23:06
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1331929386&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1331929386, 'agg_size': 100, 'api_version': '3.0', 

{'after': 1339310296, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1339310296}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 175.67, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1339310296}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 21041}
500
2012-06-19 17:10:13
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1340151013&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1340151013, 'agg_size': 100, 'api_version': '3.0', 

{'after': 1347770739, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1347770739}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 197.79, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1347770739}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 16541}
500
2012-09-28 17:28:46
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1348878526&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1348878526, 'agg_size': 100, 'api_version': '3.0', 

{'after': 1357840436, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1357840436}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 312.13, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1357840436}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 12041}
500
2013-01-19 16:34:31
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1358642071&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1358642071, 'agg_size': 100, 'api_version': '3.0', 

{'after': 1366851812, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1366851812}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 316.5, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1366851812}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 7541}
500
2013-05-05 05:28:51
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1367756931&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1367756931, 'agg_size': 100, 'api_version': '3.0', 'b

{'after': 1374957457, 'agg_size': 100, 'api_version': '3.0', 'before': 1380412800, 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'terms': {'subreddit': ['random_acts_of_pizza']}}, {'range': {'created_utc': {'gt': 1374957457}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'should': []}}, 'must_not': []}}, 'size': 500, 'sort': {'created_utc': 'asc'}}, 'execution_time_milliseconds': 255.8, 'metadata': 'true', 'ranges': [{'range': {'created_utc': {'gt': 1374957457}}}, {'range': {'created_utc': {'lt': 1380412800}}}], 'results_returned': 500, 'shards': {'failed': 0, 'skipped': 0, 'successful': 36, 'total': 36}, 'size': 500, 'sort': 'asc', 'sort_type': 'created_utc', 'subreddit': ['Random_Acts_Of_Pizza'], 'timed_out': False, 'total_results': 3041}
500
2013-08-04 06:18:44
https://api.pushshift.io/reddit/search/submission/?&size=500&after=1375622324&before=1380412800&subreddit=Random_Acts_Of_Pizza&metadata=true
{'after': 1375622324, 'agg_size': 100, 'api_version': '3.0', 'b

In [103]:
deleted_count = 0 

for k in submission_pizza.keys():
    if submission_pizza[k]['author'] == '[deleted]':
        deleted_count +=1
        
print(len(submission_pizza) - deleted_count)

26085


In [ ]:
### Testing from Tims's pizza paper -- currently numbers are not matching up
sub='science'

#before and after dates - after will keep on getting pushed back
before = "1546300800" #01/01/2019 - Rando date for checking
after = "1104537600"  #January 1st, 2005 (before reddit was founded)

submission_science = get_all_submissions(after, before, sub)